# Data Structure

- $p$: number of industries
- $k$: number of regions
- $i$: time periods index, $i = 1, \dots, n$
- Your variable $x_i$ is observed for all individuals across all regions and is a $p \times k$ cross-sectional matrix.

# Test Statistic

First, we generate the matrix $R$ defined in equation (1) of the papers. We go with some random data. 

In [230]:
import importlib

if importlib.util.find_spec('numpy') is None:
    !pip3 install numpy

if importlib.util.find_spec('scipy') is None:
    !pip3 install scipy

import numpy as np
from scipy.stats import chi2

In [231]:
n = 1 ## number of time periods

f_sq = 0

for i in range(t):
    xi = np.random.rand(2, 3) ## p*k
    p = np.shape(xi)[0] ## get p,k from data
    k = np.shape(xi)[1] ## get p,k from data
    fi = np.reshape(xi.flatten(order='F'), (p*k, 1)) ## pk*1

f_sq += np.dot(fi,fi.T) ## pk*pk

R = f_sq/n ## pk*pk

Then we stack $R$ to $\mathcal{R}(R)$ defined in equation (5).

In [232]:
R_cal = np.empty([p ** 2, k ** 2])
for i in range(1,p+1):
    for j in range(1,p+1):
        block_ij = R[((i-1)*k):(i*k), ((j-1)*k):(j*k)] # k*k     
        vector_ij = block_ij.flatten(order='F') # k^2*1
        R_cal[(j-1)*p+i-1,:] = vector_ij # the p-th row

from which we can perform the singular value decomposition (equation (9) in the paper):

In [233]:
L, Sigma_v, Nt = np.linalg.svd(R_cal)
Sigma = np.diag(Sigma_v)
N = Nt.T

length = max(np.shape(L),np.shape(N))[0]
width = min(np.shape(L),np.shape(N))[0]
diff = length - width 

happend = np.zeros([np.shape(Sigma)[0], diff])
vappend = np.zeros([diff, np.shape(Sigma)[0]])

if np.shape(L)[0] > np.shape(N)[0]:
  Sigma = np.vstack((Sigma, vappend))
elif np.shape(L)[0] < np.shape(N)[0]:
  Sigma = np.hstack((Sigma, happend))

and obtain the components of the KPST

In [234]:
L2 = np.delete(L, 0, 1)
Sigma2 = np.delete(Sigma, 0, 1)
Sigma2 = np.delete(Sigma2, 0, 0)
N2 = np.delete(N, 0, 1)

vec_R_cal = R_cal.flatten(order='F')
V = np.outer(vec_R_cal.T,vec_R_cal.T)

Now we can calculate KPST based on equation (22):

In [235]:
K1 = Sigma2.flatten(order='F') 
K2 = np.kron(N2,L2).T@V@np.kron(N2,L2)

KPST = n* K1.T @np.linalg.inv(K2) @K1

# Example

Now we have the KPST statistic in hand, we compare it with $\chi^2(df,1-\alpha)$, where the degree of freedom $df$ is calculated using equation (25).

In [236]:
## 1. Put all the previous steps in a function

def kpst(xi=np.random.rand(2, 3)):

    
   
    p = np.shape(xi)[0] ## get p,k from data
    k = np.shape(xi)[1] ## get p,k from data
   
    f_sq = 0

    for i in range(n):
        fi = np.reshape(xi.flatten(order='F'), (p*k, 1)) ## pk*1
    
    f_sq += np.dot(fi,fi.T) ## pk*pk

    R  = f_sq/n ## pk*pk
   
    R_cal = np.empty([p ** 2, k ** 2])
   
    for i in range(1,p):
     for j in range(1,p):
         block_ij = R[((i-1)*k):(i*k), ((j-1)*k):(j*k)] # k*k     
         vector_ij = block_ij.flatten(order='F') # k^2*1
         R_cal[(j-1)*p+i-1,:] = vector_ij # the p-th row
        
   L, Sigma_v, Nt = np.linalg.svd(R_cal)
   Sigma = np.diag(Sigma_v)
   N = Nt.T
   
   length = max(np.shape(L),np.shape(N))[0]
   width = min(np.shape(L),np.shape(N))[0]
   diff = length - width 
   
   happend = np.zeros([diff+1, diff])
   vappend = np.zeros([diff, diff+1])
   
   if np.shape(L)[0] > np.shape(N)[0]:
    Sigma = np.vstack((Sigma, vappend))
   elif np.shape(L)[0] < np.shape(N)[0]:
    Sigma = np.hstack((Sigma, happend))
  
   L2 = np.delete(L, 0, 1)
   Sigma2 = np.delete(Sigma, 0, 1)
   Sigma2 = np.delete(Sigma2, 0, 0)
   N2 = np.delete(N, 0, 1)
   vec_R_cal = R_cal.flatten(order='F')
   V = np.outer(vec_R_cal.T,vec_R_cal.T)

   K1 = Sigma2.flatten(order='F') 
   K2 = np.kron(N2,L2).T@V@np.kron(N2,L2)
   KPST = n* K1.T @np.linalg.inv(K2) @K1
   return KPST

## 2. Function that returns the degree of freedom
def dof(p,k):
  dof = (0.5*k*(k+1)-1)*(0.5*p*(p+1)-1)
  return dof

Let's try performing the test:

In [237]:
kpst_obs = []

## Use a loop to deal with the case of not-inversible matrix
while len(kpst_obs) < 1:
    new_value = kpst(p=5,k=7,n=1)
    if ~np.isnan(new_value):
        kpst_obs.append(new_value) 

## But there is still possibility of non-convergence of SVD, just rerun the code on error

## Test
if kpst_obs[0] < chi2.ppf(1-.05, dof(p=5,k=7)):
    print("We can't reject the null that R has KPS at nominal size 0.05. We conclude that R has KPS.")
else: 
    print("We reject the null that R has KPS at nominal size 0.05. We conclude that R doesn't has KPS.")

We can't reject the null that R has KPS at nominal size 0.05. We conclude that R has KPS.
